In [3]:
from glob import glob
import os
import torch
import torchmetrics
import nibabel as nib
import subprocess
import numpy as np

# test

In [11]:
a = [0.9886314342419307, 0.7337557004143794, 0.6706315270314614, 0.8246850349009037, 0.8168798744678497, 0.8787186359365781, 0.5866298977906506, 0.7963445919255415, 0.8573970921337605, 0.6940363133947055]
b = [0.9920792671541373, 0.8189347428580125, 0.7788211400310199, 0.885546533515056, 0.8655626592536767, 0.9174858294427395, 0.660839295387268, 0.849448365966479, 0.9004552312195301, 0.820457360893488]

c = [0.988385350595821, 0.7905999449166384, 0.6043753504753113, 0.8702174495566976, 0.8641782121224837, 0.8867943330244584, 0.6648830744353208, 0.8499266337264668, 0.9118592695756392, 0.71202500408346, 0.8485370614311912]
d = [0.991554976051504, 0.8361168996854262, 0.6591052862730893, 0.906168179620396, 0.8935618790713223, 0.9219216238368642, 0.7316537884148684, 0.8677309697324579, 0.9349090191450986, 0.8438233987851577, 0.8981850851665844]

print(np.mean(a))
print(np.mean(b))
print(np.mean(c))
print(np.mean(d))

0.784771010223776
0.8489630425721406
0.8174346985403171
0.8622482823438883


## SVBP ANTS Cross Dices

In [2]:
forward_ddfs = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', "6*_warp.nii.gz")))
bacward_ddfs = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', "6*_warp_inverse.nii.gz")))
gin_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "6*_deformable_convert.nii.gz")))
num_labels = 11
dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=num_labels)
dice_values = []
dices = []
for i in range(num_labels):
    dice_values.append([])
if len(forward_ddfs) == len(bacward_ddfs) and len(forward_ddfs) == len(gin_labels):
    for i in range(len(forward_ddfs)):
        ith_labels = torch.from_numpy(nib.load(gin_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
        for j in range(len(forward_ddfs)):
            if j != i:           
                output = 'test_labels.nii.gz'
                command = ['antsApplyTransforms']
                command.extend(['-d', '3'])
                command.extend(['-i', gin_labels[j]])
                command.extend(['-r', gin_labels[i]])
                command.extend(['-o', output])
                command.extend(['-n', 'NearestNeighbor'])
                command.extend(['-t', bacward_ddfs[i]])
                command.extend(['-t', forward_ddfs[j]])
                subprocess.call(command) 
                jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
                dice_values_ij = dicemetric(jth_labels_to_ith_labels,ith_labels)
                for k in range(len(dice_values_ij)):
                    dice_values[k].append(dice_values_ij[k].item())
    for k in range(len(dice_values)):
        dices.append(np.mean(dice_values[k]))
    print(dices)

[0.9886314342419307, 0.7337557004143794, 0.6706315270314614, 0.8246850349009037, 0.8168798744678497, 0.8787186359365781, 0.5866298977906506, 0.7963445919255415, 0.8573970921337605, 0.6940363133947055, nan]


## MAP6 ANTS Cross Dices

In [3]:
forward_ddfs = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', "M*_warp.nii.gz")))
bacward_ddfs = sorted(glob(os.path.join('dataset3', 'GIN', 'MRI', "M*_warp_inverse.nii.gz")))
gin_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "M*_deformable_convert.nii.gz")))
num_labels = 11
dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=num_labels)
dice_values = []
dices = []
for i in range(num_labels):
    dice_values.append([])
if len(forward_ddfs) == len(bacward_ddfs) and len(forward_ddfs) == len(gin_labels):
    for i in range(len(forward_ddfs)):
        ith_labels = torch.from_numpy(nib.load(gin_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
        for j in range(len(forward_ddfs)):
            if j != i:           
                output = 'test_labels.nii.gz'
                command = ['antsApplyTransforms']
                command.extend(['-d', '3'])
                command.extend(['-i', gin_labels[j]])
                command.extend(['-r', gin_labels[i]])
                command.extend(['-o', output])
                command.extend(['-n', 'NearestNeighbor'])
                command.extend(['-t', bacward_ddfs[i]])
                command.extend(['-t', forward_ddfs[j]])
                subprocess.call(command) 
                jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
                ith_labels[np.where(ith_labels == 12)] = 2
                jth_labels_to_ith_labels[np.where(jth_labels_to_ith_labels == 12)] = 2                
                dice_values_ij = dicemetric(jth_labels_to_ith_labels,ith_labels)
                for k in range(len(dice_values_ij)):
                    dice_values[k].append(dice_values_ij[k].item())
    for k in range(len(dice_values)):
        dices.append(np.mean(dice_values[k]))
    print(dices)

[0.988385350595821, 0.7905999449166384, 0.6043753504753113, 0.8702174495566976, 0.8641782121224837, 0.8867943330244584, 0.6648830744353208, 0.8499266337264668, 0.9118592695756392, 0.71202500408346, 0.8485370614311912]


## SVBP DEEP Cross Dices

In [4]:
forward_ddfs = sorted(glob(os.path.join('output', 'GIN', 'baseline-sym-ddf', 'DeformableWarp', "6*_warp.nii.gz")))
bacward_ddfs = sorted(glob(os.path.join('output', 'GIN', 'baseline-sym-ddf', 'DeformableWarp', "6*_invwarp.nii.gz")))
gin_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "6*_deformable_convert.nii.gz")))
num_labels = 11
dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=num_labels)
dice_values = []
dices = []
for i in range(num_labels):
    dice_values.append([])
if len(forward_ddfs) == len(bacward_ddfs) and len(forward_ddfs) == len(gin_labels):
    for i in range(len(forward_ddfs)):
        ith_labels = torch.from_numpy(nib.load(gin_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
        for j in range(len(forward_ddfs)):
            if j != i:           
                output = 'test_labels.nii.gz'
                command = ['antsApplyTransforms']
                command.extend(['-d', '3'])
                command.extend(['-i', gin_labels[j]])
                command.extend(['-r', gin_labels[i]])
                command.extend(['-o', output])
                command.extend(['-n', 'NearestNeighbor'])
                command.extend(['-t', bacward_ddfs[i]])
                command.extend(['-t', forward_ddfs[j]])
                subprocess.call(command) 
                jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
                dice_values_ij = dicemetric(jth_labels_to_ith_labels,ith_labels)
                for k in range(len(dice_values_ij)):
                    dice_values[k].append(dice_values_ij[k].item())
    for k in range(len(dice_values)):
        dices.append(np.mean(dice_values[k]))
    print(dices)

[0.9920792671541373, 0.8189347428580125, 0.7788211400310199, 0.885546533515056, 0.8655626592536767, 0.9174858294427395, 0.660839295387268, 0.849448365966479, 0.9004552312195301, 0.820457360893488, nan]


## MAP6 DEEP Cross Dices

In [5]:
forward_ddfs = sorted(glob(os.path.join('output', 'GIN', 'baseline-sym-ddf', 'DeformableWarp', "M*_warp.nii.gz")))
bacward_ddfs = sorted(glob(os.path.join('output', 'GIN', 'baseline-sym-ddf', 'DeformableWarp', "M*_invwarp.nii.gz")))
gin_labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "M*_deformable_convert.nii.gz")))
num_labels = 11
dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=num_labels)
dice_values = []
dices = []
for i in range(num_labels):
    dice_values.append([])
if len(forward_ddfs) == len(bacward_ddfs) and len(forward_ddfs) == len(gin_labels):
    for i in range(len(forward_ddfs)):
        ith_labels = torch.from_numpy(nib.load(gin_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
        for j in range(len(forward_ddfs)):
            if j != i:           
                output = 'test_labels.nii.gz'
                command = ['antsApplyTransforms']
                command.extend(['-d', '3'])
                command.extend(['-i', gin_labels[j]])
                command.extend(['-r', gin_labels[i]])
                command.extend(['-o', output])
                command.extend(['-n', 'NearestNeighbor'])
                command.extend(['-t', bacward_ddfs[i]])
                command.extend(['-t', forward_ddfs[j]])
                subprocess.call(command) 
                jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
                ith_labels[np.where(ith_labels == 12)] = 2
                jth_labels_to_ith_labels[np.where(jth_labels_to_ith_labels == 12)] = 2  
                dice_values_ij = dicemetric(jth_labels_to_ith_labels,ith_labels)
                for k in range(len(dice_values_ij)):
                    dice_values[k].append(dice_values_ij[k].item())
    for k in range(len(dice_values)):
        dices.append(np.mean(dice_values[k]))
    print(dices)

[0.991554976051504, 0.8361168996854262, 0.6591052862730893, 0.906168179620396, 0.8935618790713223, 0.9219216238368642, 0.7316537884148684, 0.8677309697324579, 0.9349090191450986, 0.8438233987851577, 0.8981850851665844]


## SVBP ANTSMean Cross Dices

In [5]:
forward_ddfs = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'MRI', "6*_warp.nii.gz")))
bacward_ddfs = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'MRI', "6*_warp_inverse.nii.gz")))
gin_labels = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'Labels', "6*_deformable_convert.nii.gz")))
num_labels = 11
dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=num_labels)
dice_values = []
dices = []
for i in range(num_labels):
    dice_values.append([])
if len(forward_ddfs) == len(bacward_ddfs) and len(forward_ddfs) == len(gin_labels):
    for i in range(len(forward_ddfs)):
        ith_labels = torch.from_numpy(nib.load(gin_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
        for j in range(len(forward_ddfs)):
            if j != i:           
                output = 'test_labels.nii.gz'
                command = ['antsApplyTransforms']
                command.extend(['-d', '3'])
                command.extend(['-i', gin_labels[j]])
                command.extend(['-r', gin_labels[i]])
                command.extend(['-o', output])
                command.extend(['-n', 'NearestNeighbor'])
                command.extend(['-t', bacward_ddfs[i]])
                command.extend(['-t', forward_ddfs[j]])
                subprocess.call(command) 
                jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
                dice_values_ij = dicemetric(jth_labels_to_ith_labels,ith_labels)
                for k in range(len(dice_values_ij)):
                    dice_values[k].append(dice_values_ij[k].item())
    for k in range(len(dice_values)):
        dices.append(np.mean(dice_values[k]))
    print(dices)

[0.9904695721964042, 0.8342637764910857, 0.8319898342092832, 0.8877245374023914, 0.8548560688893, 0.9057215678195158, 0.6491817720234394, 0.8733555113275846, 0.9083404324948787, 0.8345103286206722, nan]


## MAP6 ANTSMean Cross Dices

In [6]:
forward_ddfs = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'MRI', "M*_warp.nii.gz")))
bacward_ddfs = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'MRI', "M*_warp_inverse.nii.gz")))
gin_labels = sorted(glob(os.path.join('dataset3', 'GIN_Mean', 'Labels', "M*_deformable_convert.nii.gz")))
num_labels = 11
dicemetric = torchmetrics.Dice(average=None, mdmc_average='global', num_classes=num_labels)
dice_values = []
dices = []
for i in range(num_labels):
    dice_values.append([])
if len(forward_ddfs) == len(bacward_ddfs) and len(forward_ddfs) == len(gin_labels):
    for i in range(len(forward_ddfs)):
        ith_labels = torch.from_numpy(nib.load(gin_labels[i]).get_fdata().ravel()).type('torch.LongTensor')
        for j in range(len(forward_ddfs)):
            if j != i:           
                output = 'test_labels.nii.gz'
                command = ['antsApplyTransforms']
                command.extend(['-d', '3'])
                command.extend(['-i', gin_labels[j]])
                command.extend(['-r', gin_labels[i]])
                command.extend(['-o', output])
                command.extend(['-n', 'NearestNeighbor'])
                command.extend(['-t', bacward_ddfs[i]])
                command.extend(['-t', forward_ddfs[j]])
                subprocess.call(command) 
                jth_labels_to_ith_labels = torch.from_numpy(nib.load(output).get_fdata().ravel()).type('torch.LongTensor')
                ith_labels[np.where(ith_labels == 12)] = 2
                jth_labels_to_ith_labels[np.where(jth_labels_to_ith_labels == 12)] = 2                
                dice_values_ij = dicemetric(jth_labels_to_ith_labels,ith_labels)
                for k in range(len(dice_values_ij)):
                    dice_values[k].append(dice_values_ij[k].item())
    for k in range(len(dice_values)):
        dices.append(np.mean(dice_values[k]))
    print(dices)

[0.9902225055477836, 0.8518304234201258, 0.7087382636286995, 0.9160725463520397, 0.899229907989502, 0.9174034671349959, 0.710911258242347, 0.8893685552206907, 0.9382049560546875, 0.8234538679773157, 0.8730658812956377]
